# Run obstacle case by PyRunOF. 

The test was created to explain the general options and possibility of the python libraries PyRunOF. 

The *test* consist of 
    
* **settings** folder is the folder to be stored a setting to run simulation.
* **solution** folder is the folder to be stored solution of simulation.
* **compute.py** is the general python script to run the case in parametric sweep mode.
* **test_lib.ipynb** is the python notebook considering procedures of simulation of obstacle case.
* **test_lib.py** is the python script to run simulation of obstacle case.

Let's start to import pyRunOF libraries and input data from data.py in settings folder. 
The data.py consist of parameters describing the modeling system. For example, geometry, mesh and other parameters. 

In [2]:
import pyRunOF
from settings.data import *

To make easy our process to work with directories we initialize a python object. The oject is created by Manipulation class.
We make a link to the object as **mp**. In the future the Manipulation object will be able as **mp**. 

In [3]:
mp = pyRunOF.Manipulations(dir_path=dir_path)

To work with settings of our case we will often response to files in **settings** and **solutions** folders. 
Therefore we create variables to store the pathes using method **create_path_dir** from **mp** object. 
**NOTICE:** the method **create_path_dir** does not make a folder and create and store only string with requre path to the folder. By the way the folder specified to the path might be not existed. 

In [4]:
mp.create_path_dir(dir_path_key='dir', case_name='settings',
                    path_key='settings')
mp.create_path_dir(dir_path_key='dir', case_name='solution',
                    path_key='solution')

Let us to specify a path to the folder containing settings of an openfoam case. In the next steps we will response for name of the openfoam case. Therefore it is usefull to create the name and store the information in our code. In order to realize it we use **create_name** method. Then crate path of the case with built naem case from previos procedure. 

In [5]:
mp.create_name(name_base=src_case, name_key=src_name_key)
mp.create_path_dir(dir_path_key='settings', name_key=src_name_key,
                    path_key=src_path_key)

We will run openfoam case in **solution** folder. The name of case folder specify by method **create_name** and then create path to the case.  

In [7]:
mp.create_name('test', name_base=src_case, name_key=dst_name_key)
mp.create_path_dir(dir_path_key='solution', name_key=dst_name_key,
                    path_key=dst_path_key)

In the test we store some parametrs in json file. The case contains parameters of mesh and others. Let us to specify the path as previos step. 

In [8]:
# path to json parameters
mp.create_path_dir(dir_path_key='settings', case_name='parameters.json',
                    path_key='parameters_path')
# path to json parameters of mesh
mp.create_path_dir(dir_path_key='settings', case_name='mesh_parameters.json',
                    path_key='mesh_parameters_path')

To make work easier we collect all parameters describing numerical model and auxilary options in one json file by **create_json_params**. These parameters are collected from python dictionaries defined as in **data.py** of **settings** folder and so json files **parameters.json**   and **mesh_parameters.json**.

In [9]:
# collect general dict of parameters и записывает в путь по ключу parameters_path
mp.create_json_params(time_dict, parallel_dict, prop_dict, zero_dict, library_path,
                        mp.get_dict_from_json(mp.get_path('mesh_parameters_path')),
                        save_path=mp.get_path('parameters_path'))


The test conducts build a mesh by salome. All instrutions to generate mesh in salome and transform to openfaom mesh format are listed in a python script. 
In order for the test to run the script we need specify the path to one. First of all create the name of the script and then the path. 

Here we create poly_mesh_path to save prepared mesh from salome mesh script to the path. The aprroach is temporary and in the further version of PyRunOF will be implemented in a method.

In [19]:
# имя скрипта для создания сетки
mp.create_name(name_base='create_obstacle_mesh.py', only_base=True, name_key='salome_script')
# путь к скрипту сетки
mp.create_path_dir(dir_path_key='settings', name_key='salome_script', path_key='salome_script_path')

poly_mesh_path = mp.get_constant_path(str(mp.get_path(dst_path_key))) / 'polyMesh'

We will use **data** dictonary to store all prameters. Let us to initialze the dictionary. 
And then create json file with all these parameters of the dictionary. 

In [ ]:
data = mp.get_dict_from_json(mp.get_path('parameters_path'))

data.update({'constant_path': str(poly_mesh_path)})
# collect general dict of parameters и записывает в путь по ключу parameters_path
mp.create_json_params(data, save_path=mp.get_path('parameters_path'))

In the current step we copy of base openfoam case from setting folder to solution foler with new name folder as specfied above. If the solution folder not exist we create the folder by **create_folder** method. 

In [11]:
# if the solution folder does not exist we create it by below written code
if not os.path.exists(mp.get_path('solution')):
    mp.create_folder(dir_key='dir', folder_name='solution')

mp.duplicate_case(src_key=src_path_key, dist_key=dst_path_key, mode='rewrite')

To process files in system folder of openfoam case we initialize system object by **System** class of PyRunOF library. 
Method **set_controlDict** find text parts in *controlDict* file as keys of **data** dictonary and replace it on values corresponding the keys. 

In [13]:
system = pyRunOF.System(case_path=mp.get_path(dst_path_key))
system.set_controlDict(data)

To process files in constant folder of openfoam case we initialize system object by **Constant** class of PyRunOF library. 
Method **set_transportProp** find text parts in *transportProp* file as keys of **data** dictonary and replace it on values corresponding the keys. 

In [14]:
constant = pyRunOF.Constant(case_path=mp.get_path(dst_path_key), lib_path=library_path['lib_path_var'])
constant.set_transportProp(data)

In the part of the test we prepapre settings for initial values of the model. Therefore we need initialize **InitialValues** class and write obtained object in zero variable.  

The method **set_var** is method to replace found text tempalte as key of data dict on its values. 

In [17]:
########### Initial conditions #####################
zero = pyRunOF.InitialValue(case_path=mp.get_path(dst_path_key))

zero.set_var(data,)

Here we intialize *mesh* object using **Mesh** class. Then set the parameters for decompsoePar procedure. The procedure conducts deviding mesh into parts for parallel calculations. 

In [20]:
########### mesh settings #####################
mesh = pyRunOF.Mesh(case_path=mp.get_path(dst_path_key))
mesh.set_decomposePar(data)

Now we can run salome mesh script to build mesh for our model. 

In [7]:
mesh.run_salome_mesh(script_path=mp.get_path('salome_script_path'),
                        parameter_path=mp.get_path('parameters_path'))

bash: cannot set terminal process group (24824): Inappropriate ioctl for device
bash: no job control in this shell


/home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/settings
"ViscousLayers2D=0.00086...,7,1.03,[10, 3],0" was not assigned to "final geometry" shape in "final geometry" mesh  : hypothesis of the same dimension is already assigned to this shape
Number of nodes: 51896

Number of cells: 25475

Counting number of faces:

total number of faces: 102373, internal: 50355, external 52018

Converting mesh to OpenFOAM

Finished processing boundary faces

Finished processing volumes.

Sorting faces in upper triangular order

Writing the file points

Writing the file faces

Writing the file owner

Writing the file neighbour

Writing the file boundary

Writing file cellZones

Finished writing to /home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/settings//home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/solution/obstacle_base_test/constant/polyMesh 

Converted mesh in 2s

Wrote mesh in 1s

Total time: 3s


Here we run procedure of mesh decomposition 

In [8]:
mesh.run_decompose(what='OF')

bash: cannot set terminal process group (24824): Inappropriate ioctl for device
bash: no job control in this shell


/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     | Website:  https://openfoam.org
    \\  /    A nd           | Version:  6
     \\/     M anipulation  |
\*---------------------------------------------------------------------------*/
Build  : 6-47517f2ebb1b
Exec   : decomposePar -force
Date   : Jan 25 2024
Time   : 14:35:50
Host   : "ivan"
PID    : 25227
I/O    : uncollated
Case   : /home/ivan/programms/OpenFOAM/PyRunOF/tests/obstacle_new_ps/solution/obstacle_base_test
nProcs : 1
sigFpe : Enabling floating point exception trapping (FOAM_SIGFPE).
fileModificationChecking : Monitoring run-time modified files using timeStampMaster (fileModificationSkew 10)
allowSystemOperations : Allowing user-supplied system call operations

// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
Create time

Removing 0 existi

To run our model for calculation we will be able to use object of **Run** classes. Let us initialize it and set some settings. 
We want to save log file with information of solution, provide prallel calculation on the number of **data['core_OF']** corecores.

In [ ]:
runner = pyRunOF.Run(solver='pisoFoam', path_case=mp.get_path(dst_path_key))
runner.set_log_flag(log_flag=True)
runner.set_mode(mode='parallel')
runner.set_cores(coreOF=data['core_OF'])

Here we run our model for calculation!!!

In [ ]:
runner.run()
